In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 扩展卡尔曼滤波（Extended Kalman Filter，EKF）

在卡尔曼滤波中，要满足两个条件：观测是状态的线性函数，下一个状态是当前状态的线性函数。这两个假设对KF的正确性是很重要的。

不幸的是，实际上状态转移函数和测量函数很少是线性函数。例如具有恒定线速度和角速度的机器人的典型移动轨迹是圆。对于这种情况有两种方案：
1. 将状态转移函数和测量函数进行线性化，利用参数化的分布来保存不确定性。
2. 利用非参数的方法，例如直方图或粒子集的方法来保存分布的不确定性。

## 1. 扩展卡尔曼滤波算法

### 1.1 非线性的状态转移函数和测量函数

令非线性的状态转移函数为：
$$x_t = g(u_t, x_{t-1} ) + \varepsilon_t$$

非线性的观测函数为：
$$z_t = h(x_t) + \delta_t$$

准确的实现置信度更新对于非线性函数$g$和$h$通常是不可能的，贝叶斯滤波不存在闭式解。EKF计算**真实置信度的高斯近似值**，因此EKF表示时刻$t$的置信度$bel(x_t)$，具有均值$\mu_t$和协方差$\Sigma_t$。

### 1.2 通过泰勒展式进行线性化

泰勒展开根据$g$的值和斜率构造一个函数$g$的线性近似函数，斜率由下面的偏导数给出：
$$G_t = g'(u_t, x_{t-1}) := \frac {\partial g(u_t, x_{t-1})}{\partial x_{t-1}}$$
$G_t$是函数$g$的Jacobian。

用相同的方法可以求得测量函数的Jacobian：
$$H_t = h'(x_t) = \frac {\partial h(x_t)}{\partial x_t}$$

### 1.3 扩展卡尔曼滤波算法

----
1：**Algorithm Extended_Kalman_filter**$( \mu_{t-1}, \Sigma_{t-1}, u_t, z_t )$**:**  
2：&emsp;&emsp;$\overline \mu_t = g(u_t, \mu_{t-1})$  
3：&emsp;&emsp;$\overline \Sigma_t = G_t \Sigma_{t-1}G_t^T + R_t$  
4：&emsp;&emsp;$K_t = \overline \Sigma_t H_t^T ( H_t \overline \Sigma_t H_t^T + Q_t)^{-1}$  
5：&emsp;&emsp;$\mu_t = \overline \mu_t + K_t (z_t - h(\overline \mu_t))$  
6：&emsp;&emsp;$\Sigma_t = (I - K_t H_t) \overline \Sigma_t$  
7：&emsp;&emsp;**return** ${\mu_t, \Sigma_t}$  

----

## 2. 仿真模型

假定有一个在平面环境操作的移动机器人，其状态是它的$(x,y)^T$位置和它的全局航向$\theta$。假定已知确切的$x$和$y$，但是方向$\theta$是未知的。例如初始的估计为：
$$\mu = (0, 0, \frac {\pi}{4}) \qquad \Sigma = \begin{bmatrix}
0.01 & 0 & 0 \\ 0 & 0.01 & 0 \\ 0 & 0 & 10000
\end{bmatrix}$$

假设机器人的运动控制是向前移动$u_t$个单位，因此移动后机器人的期望位置将为：
$$\begin{bmatrix}
x' \\ y' \\ \theta'
\end{bmatrix} = \begin{bmatrix}
x + u_t cos \theta \\ y + u_t sin \theta \\ \theta
\end{bmatrix}$$

测量量是机器人距离世界坐标系原点的距离。
$$z_t = h(x_t) = \sqrt{x^2 + y^2} $$

### 2.1 状态转移函数的定义

In [2]:
def state_transfer_function(mu, u):
    x0, y0, theta0 = np.squeeze(mu)
    x1 = x0 + u * np.cos(theta0)
    y1 = y0 + u * np.sin(theta0)
    theta1 = theta0
    return np.array([x1, y1, theta1]).reshape((3, 1))

state_transfer_function(np.array([0, 0, np.pi / 6]).reshape((3, 1)),5)

array([[ 4.33012702],
       [ 2.5       ],
       [ 0.52359878]])

### 2.2 测量函数的定义

In [3]:
def measurement_function(mu):
    x0, y0, theta0 = np.squeeze(mu)
    z = np.sqrt(x0 ** 2 + y0 ** 2)
    return np.array([z]).reshape((1,1))

measurement_function(np.array([3, 4, np.pi / 6]).reshape((3, 1)))

array([[ 5.]])

### 2.3 状态转移采样函数（带噪声）

In [4]:
def state_transfer_sample(mu, u, Sigma_u):
    x0, y0, theta0 = np.squeeze(mu)
    noise_x, noise_y, noise_theta = np.random.multivariate_normal(np.zeros(3), Sigma_u)
    x1 = x0 + noise_x + u * np.cos(theta0 + noise_theta)
    y1 = y0 + noise_y + u * np.sin(theta0 + noise_theta)
    theta1 = theta0 + noise_theta
    return np.array([x1, y1, theta1]).reshape((3, 1))

Sigma = np.diag([0.2**2, 0.2 ** 2, 0.05 ** 2])
state_transfer_sample(np.array([0, 0, np.pi / 6]).reshape((3, 1)), 5, Sigma)

array([[ 4.19173115],
       [ 2.55247488],
       [ 0.46528393]])

### 2.4 测量采样函数（带噪声）

In [5]:
def measurement_sample(mu, Sigma_z):
    x0, y0, theta0 = np.squeeze(mu)
    noise = np.random.normal(0, Sigma_z)
    z = np.sqrt(x0 ** 2 + y0 ** 2) + noise
    return np.array([z]).reshape((1,1))

measurement_sample(np.array([1, 1, np.pi / 6]).reshape((3, 1)), Sigma_z=0.1)

array([[ 1.53528704]])

### 2.5 状态转移函数的Jacobian矩阵

In [6]:
def jacobian_G(mu, u):
    x, y, theta = np.squeeze(mu)
    G = np.eye(3,dtype=np.float64)
    G[0][2] = - u * np.cos(theta)
    G[1][2] = u * np.cos(theta)
    return G

jacobian_G(np.array([1, 1, np.pi / 6]).reshape((3, 1)), 1)

array([[ 1.       ,  0.       , -0.8660254],
       [ 0.       ,  1.       ,  0.8660254],
       [ 0.       ,  0.       ,  1.       ]])

### 2.6 测量函数的Jacobian矩阵

In [7]:
def jacobian_H(mu):
    x, y, theta = np.squeeze(mu)
    distance = np.sqrt(x ** 2 + y ** 2)
    H = np.array([x/distance, y / distance, 0]).reshape((1, 3))
    return H

# jacobian_H( np.array([1, 1, np.pi / 6]).reshape((3, 1)) )

### 2.7 扩展卡尔曼滤波算法

In [8]:
def extended_kalman_filter(mu0, Sigma0, u, z, sigma_u=0.2, sigma_z=0.1):
    # 重新规整输入参数的形状
    mu0 = np.array(mu0).reshape((3, 1))
    Sigma0 = np.array(Sigma0).reshape((3,3))
    z = np.array(z).reshape((1,1))
    
    # 控制方程和测量方程的误差协方差矩阵
    R = np.eye(3, dtype=np.float64) * sigma_u ** 2
    Q = np.array([1]).reshape((1,1)) * sigma_z ** 2
    
    # 计算雅可比矩阵
    G = jacobian_G(mu, u)
    H = jacobian_H(mu)
    
    # 执行控制预测步骤
    mu1 = state_transfer_function(mu0, u)
    Sigma1 = np.dot(np.dot(G, Sigma0), G.T) + R
    
    # 进行测量更新步骤
    Temp = np.linalg.inv(np.dot(np.dot(H, Sigma1), H.T) + Q)
    K = np.dot(np.dot(Sigma1, H.T), Temp)
    mu2 = mu1 + np.dot(K, (z - measurement_function(mu1)))
    Sigma2 = np.dot((np.eye(3) - np.dot(K, H)), Sigma1)
    
    return mu2, Sigma2

mu = np.array([1, 1, np.pi/4]).reshape((3, 1))
Sigma = np.diag([0.01, 0.01, 0.1])
u = 1
z = 2.42
mu ,Sigma = extended_kalman_filter(mu, Sigma, u, z, sigma_u=0.2, sigma_z=0.1)
print(Sigma)
u = 1
z = 3.42
mu ,Sigma = extended_kalman_filter(mu, Sigma, u, z, sigma_u=0.2, sigma_z=0.1)
print(Sigma)
u = 1
z = 3.42
mu ,Sigma = extended_kalman_filter(mu, Sigma, u, z, sigma_u=0.2, sigma_z=0.1)
print(Sigma)

[[ 0.07916667 -0.07083333 -0.07071068]
 [-0.07083333  0.07916667  0.07071068]
 [-0.07071068  0.07071068  0.14      ]]
[[ 0.26914286 -0.26085714 -0.16970563]
 [-0.26085714  0.26914286  0.16970563]
 [-0.16970563  0.16970563  0.18      ]]
[[ 0.61914216 -0.61085784 -0.29698485]
 [-0.61085784  0.61914216  0.29698485]
 [-0.29698485  0.29698485  0.22      ]]
